In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from sklearn import preprocessing
# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
TRAIN_PATH = '../input/train.csv'
# trainDF =  pd.read_csv('../input/train.csv', chunksize = 100000)
# trainDF = trainDF.get_chunk(100000)

In [ ]:
%%time
# Assume we only know that the csv file is somehow large, but not the exact size
# we want to know the exact number of rows

# Method 1, using file.readlines. Takes about 20 seconds.
with open(TRAIN_PATH) as file:
    n_rows = len(file.readlines())

print ('Exact number of rows: '+str(n_rows))

In [ ]:
chunksize = 5000000 # 5 million rows at one go. Or try 10 million
total_chunk = n_rows // chunksize + 1

print('Total chunks required:'+str(total_chunk))

In [ ]:
traintypes = {'fare_amount': 'float32',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

cols = list(traintypes.keys())

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe
i=0

for df_chunk in pd.read_csv(TRAIN_PATH, usecols=cols, dtype=traintypes, chunksize=chunksize):
    
    i = i+1
    # Each chunk is a corresponding dataframe
     
    
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
    df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    
    # Can process each chunk of dataframe here
    # clean_data(), feature_engineer(),fit()
    
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk) 

In [ ]:
trainDF = pd.concat(df_list)
trainDF=trainDF.dropna()

In [ ]:
testDF =  pd.read_csv('../input/test.csv')

In [ ]:
trainDF.head()

In [ ]:
testDF.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [ ]:
y = trainDF["fare_amount"]
# y_test = testDF["fare_amount"]

In [ ]:
y.head()

In [ ]:
trainDF.columns

In [ ]:
X = trainDF[[  'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count']]
X_pred = testDF[[  'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X.head()

In [ ]:
lab_enc = preprocessing.LabelEncoder()

y_train = lab_enc.fit_transform(y_train)
y_test = lab_enc.fit_transform(y_test)

In [ ]:

model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:
# from xgboost import XGBRegressor

# my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
# my_model.fit(X_train, y_train, verbose=False)
# my_model.score(X_test,y_test)

In [ ]:
# testDF1 = testDF[[  'pickup_longitude',
#        'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
#        'passenger_count']]

In [ ]:
y_pred = my_model.predict(X_pred)

In [ ]:
y_pred.size

In [ ]:
submission = pd.DataFrame({
        "key": testDF['key'],
        "fare_amount": y_pred.round(3)
})

In [ ]:
submission.to_csv('submit.csv', index = False)